# Object Recognition Using Tensorflow Lite with MobilenetV2 Prebuild Model
**Full Script At [Object_Recognition_(TensorFlow_Lite_Edition).py](Object_Recognition_[TensorFlow_Lite_Edition].py)**
<br>
This is the documentation for Object Recognition with Tensorflow Lite using prebuild model (MobilenetV2)

***Libraries Used In The Script***
- tflite_runtime 
- OpenCV
- numpy 
- Picamera2 
- libcamera 
- os
- time 
- RPi_Robot_Hat_Lib

## Installing Libraries @ Dependencies 

- Picamera2 
- libcamera
- os
- time 
- tflite_runtime 
- OpenCV 
- numpy 
- RPi_Robot_Hat_Lib

### First, let's install the libraries ! 
1. tflite_runtime 
    - Follow the link [Tensorflow.org](https://www.tensorflow.org/lite/guide/python)
2. OpenCV 
    - Follow the link [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
3. NumPy 
    - Follow the link [numpy.org](https://numpy.org/install/)


### Verify the installed libraries 
1. tflite_runtime 
    - In Python File type: <br>
    `import tflite_runtime.interpreter as tflite` 
    - Installation sucess if codeing run without error  
2. OpenCV 
    - In Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed  
3. NumPy 
    - In Python File type: <br>
    `import numpy as np`
    - Installation sucess if codeing run without error 

## Downloading The model Files 
The Object Detection Model used in this script is: <br>
***MobilenetV2(tflite)***<br>
Download the object detection Model from github
1. Open the terminal in Raspberry Pi 
2. Type the command bellow:
- `git clone https://github.com/raspberrypi/picamera2` <br>
This will download the Picamera2 repository from Github 
3. Nevigate to the Model folder:<br>
picamera2 --> examples --> tensorflow
4. Multiple Model is included in the folder 
5. In this script the Model used is ***mobilenet_v2.tflite***
6. Beside the Model, The label is also needed.
7. The labels used for this script is ***coco_labels.txt***
8. Copy the two target files above to the same directory of the script. 


## Let's Start Coding ! 
### 1. Import all the libraries 
- Tensorflow Lite 
- OpenCV 
- NumPy 
- Picamera2 
- libcamera
- os
- time 
- RPi_Robot_Hat_Lib

In [ ]:
## Tensorflow Lite Library
import tflite_runtime.interpreter as tflite

## For Image processing 
import cv2
import numpy as np

## To capture the frames from the camera
from picamera2 import Picamera2
## Control the Auto Focus of the camera 
from libcamera import controls

## For validation of the model and label files 
import os

## To get the accurate time 
import time

## Control the Servo Pan Tilt HAT 
from RPi_Robot_Hat_Lib import RobotController 

### 2. Verify and import the model file and label files
- Check if the script is able to find the files using the os library 
- Exit the script if the model and label files are not found

****In this script all the Model and lebel file are placed under a folder called *tebsorflow_lite_examples***
<br>
***Thus, the mode_folder variable is created.***

In [ ]:
# Verify the model and label files
model_folder = 'tensorflow_lite_examples'
model_file = '/mobilenet_v2.tflite'
model_path = model_folder + model_file
label_file = '/coco_labels.txt'
label_path = model_folder + label_file

if not os.path.exists(model_path):
    print("Model file not found")
    print("Please check the path: ", model_path)
    exit()
else:
    print("Model file found at:", model_path)

if not os.path.exists(label_path):
    print("Label file not found")
    print("Please check the path: ", label_path)
    exit()
else:
    print("Label file found at:", label_path)

### 3. Read the label file and convert it to match the model class 
- The code bellow will open the coco_label.txt in read mode 
- It will read through all the lines and saperate the class ID and class name 

In [ ]:
with open(label_path, 'r') as f:
    lines = f.readlines()
    labels = {int(line.strip().split(maxsplit=1)[0]): line.strip().split(maxsplit=1)[1] for line in lines}

### 4. Set the Position Of the Pan tilt 
- Initialise the RobotController (From RPi_Robot_Hat_Lib) Library.
- Define the chanel for the vertical and horizontal servo channel.
- Using the RobotController (From RPi_Robot_Hat_Lib) library control the position of the Pan Tilt Hat . 

In [ ]:

## Initialise the Motor Controler Library 
Motor = RobotController() 

##  Control the Pna Tilt HAT using the motor controller
# Set PanTilt and servo channels
vertical = 2
horizontal = 1
Motor.set_servo(vertical, 80)
Motor.set_servo(horizontal, 90)

### 5. Initialise the Picamera2 library and start the camera 
- Initialise the Picamera2 Library and configure it to output the video format as:
    - Color Format: "XRGB8888" 
    - Frame Size: "640(Width), 480(Height)"
- The code `cam.set_controls({"AfMode": controls.AfModeEnum.Continuous})` will enable the Auto Focus function <br>
***Note**: The auto focus function only available on Raspberry Pi Camera 3 and above. Raspberry Pi Camera 1 and 2 does not support the Auto Focus Function. 

In [ ]:
# Start the camera
frame_height = 480
frame_width = 640
cam = Picamera2()
cam.configure(cam.create_preview_configuration(main={"format": 'XRGB8888', "size": (frame_width, frame_height)}))
cam.start()
cam.set_controls({"AfMode": controls.AfModeEnum.Continuous})

### 6. Define `object_detection` function 

#### Initialise the Tensorflow Lite Library 
- Point the derectory of Model path to TensorFlow Lite <br>
`interpreter = tflite.Interpreter(model_path=model_path)`
- Initialise the tensorflow Library 
    - Set the *input_details*
    - Set the *output_details*
    - Set the *height* of object detection
    - Set the *width* of object detection  

In [ ]:
def object_detection():
    interpreter = tflite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

### 7. Normalise and Insert the data To TensorFlow Lite for Detection 
 
- Obtained the captured frame from the camera and store to the variable *frame* <br>
    ```py
    frame = cam.capture_array()
    ```
- Obtain the time and store it in the t_start variable for FPS calculation<br>
    ```py 
    t_start = time.time()
    ```
- Initalise the variable *fps* <br>
    ```py 
    fps = 0
    ```
- Convert the color format of the captured from *RGB* to *BGR* using OpenCV<br>
    ```py 
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    ``` 
- Resize the frame to fit the object detection algrothim and pass it to tensorflow lite of object dedtection<br>
    ```py
    resized_frame = cv2.resize(rgb_frame, (width, height))
    input_data = np.expand_dims(resized_frame, axis=0) 
    ```
- Nomalize and Convert the data to ready for TensorFlow Lite Object Recognition 
    ```py 
    if input_details[0]['dtype'] == np.float32:
        input_data = (np.float32(input_data) - 127.5) / 127.5
    ``` 
- Insert the data to TensorFlow Lite and start the detection process 
    ```py
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    ```

In [ ]:
while True:
        frame = cam.capture_array()
        t_start = time.time()
        fps = 0
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        resized_frame = cv2.resize(rgb_frame, (width, height))
        input_data = np.expand_dims(resized_frame, axis=0)

        if input_details[0]['dtype'] == np.float32:
            input_data = (np.float32(input_data) - 127.5) / 127.5

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()


### 8. Obtain the Detected data (Object) 
- Get the coordinates of the detected Object 
```py
    detected_boxes = interpreter.get_tensor(output_details[0]['index'])[0]
```
- Get the Class ID of the detected Object 
```py
    detected_classes = interpreter.get_tensor(output_details[1]['index'])[0]
```
- Get the Score of the detected object (Similarity)
```py
    detected_scores = interpreter.get_tensor(output_details[2]['index'])[0]
```
- Get the number of object detected in a frame 
```py
    num_boxes = int(interpreter.get_tensor(output_details[3]['index'])[0])
```


In [ ]:
        detected_boxes = interpreter.get_tensor(output_details[0]['index'])[0]
        detected_classes = interpreter.get_tensor(output_details[1]['index'])[0]
        detected_scores = interpreter.get_tensor(output_details[2]['index'])[0]
        num_boxes = int(interpreter.get_tensor(output_details[3]['index'])[0])

### 9. For loop (For all Detected Object)
- Only execute if the score of the detected object is more than 0.5 (50%)
```py
    if detected_scores[i] > 0.5:
```
- Extract the coordinates from the detected_box variable 
```py
    ymin, xmin, ymax, xmax = detected_boxes[i]
```
- Get the hight and width of the detected object 
```py
    im_height, im_width, _ = frame.shape
```
- Calculate the Relative coordinates of the detected object 
```py
    left = int(xmin * im_width)
    right = int(xmax * im_width)
    top = int(ymin * im_height)
    bottom = int(ymax * im_height)   
    center_x = int((left + right) // 2)
    center_y = int((top + bottom) // 2)
```
- From the calculated coordinates using OpenCV, Draw the a rectangle box 
```py
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)  
```

- Ensure the class Id is in the label list and find the name in the label file 
```py
    class_id = int(detected_classes[i])
    if class_id in labels:
        label = labels[class_id]
    else:
        label = 'Unknown' 
```
- Display the name of the detected object 
```py
    print(f"Detected class ID: {class_id}, Label: {label}, Score: {detected_scores[i]}") 
    cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)  
```
- Calculate the FPS of the script and display the frame 
```py
    fps += 1
    mfps = fps / (time.time() - t_start)
    cv2.putText(frame, "FPS : " + str(int(mfps)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow("main", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    

```


In [ ]:
        for i in range(num_boxes):
           if detected_scores[i] > 0.5:
               ymin, xmin, ymax, xmax = detected_boxes[i]
               im_height, im_width, _ = frame.shape
               left = int(xmin * im_width)
               right = int(xmax * im_width)
               top = int(ymin * im_height)
               bottom = int(ymax * im_height)   
               center_x = int((left + right) // 2)
               center_y = int((top + bottom) // 2)
               print("Coordinates: ", "\nX: ", center_x, "\nY: ", center_y)
               cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)   
               # Ensure detected class index is within the range of labels
               class_id = int(detected_classes[i])
               if class_id in labels:
                   label = labels[class_id]
               else:
                   label = 'Unknown'    
               # Debugging output to verify label and class index
               print(f"Detected class ID: {class_id}, Label: {label}, Score: {detected_scores[i]}") 
               cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)   
        fps += 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS : " + str(int(mfps)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.imshow("main", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break    


### 10. Run Object Detection Funuction 
- When the script start run object detection function <br>
`object_detection()` 
- When Keyboard Inturrupt (Ctrl + C)
    - Stop the camera 
    - Reset the Pan Tilt Hat to default position  
    - exit the script 


In [ ]:
try:
    if __name__ == '__main__':
        object_detection()
except KeyboardInterrupt:
    cam.close()
    Motor.cleanup()
    print("Exiting")
    exit()
